# Introduction to Interactive Analysis with Spark

## Table of Content
1. [Initialization](#1.-Initialization)
2. [Creating a RDD](#2.-Creating-a-RDD)
3. [Getting Help](#3.-Getting-Help)
4. [Action on a Dataset](#4.-Action-on-a-Dataset)
5. [Dataset Transformation](#5.-Dataset-Transformation)
6. [Caching a Dataset](#6.-Caching-a-Dataset)
7. [Filtering a Dataset](#7.-Filtering-a-Dataset)
8. [Reduction Operation](#8.-Reduction-Operation)  
  8.1 [Transforming in Key-Value Pairs](#8.1-Transforming-in-Key-Value-Pairs)  
  8.2 [Filtering Unrelated Entries](#8.2-Filtering-Unrelated-Entries)  
  8.3 [Aggregating the Results by Key](#8.3-Aggregating-the-Results-by-Key)  
  8.4 [Computing the Most Frequent Languages](#8.4-Computing-the-Most-Frequent-Languages)    
  8.5 [Bar Chart](#8.5.-Bar-Chart)
9. [Ending the Analysis](#9.-Ending-the-Analysis)
10. [Recap](#10.-Recap)


## List of Exercises
1. [Exercise 1: How to Cache?](#Exercise-1)
2. [Exercise 2: How to Peek?](#Exercise-2)
3. [Exercise 3: How to Filter?](#Exercise-3)
4. [Exercise 4: How to Sort?](#Exercise-4)

## 1. Initialization

In this notebook, we will use Spark to analyze brievly a simple structured dataset.

First, we need to import Spark's Python module named `pyspark`.

In [ ]:
import pyspark
import spark_helper # homemade module to ease the workshop progress

Then, we need to create a SparkContext.

In [ ]:
try:
    sc = pyspark.SparkContext()
except ValueError:
    print("Warning : a SparkContext already exists.")

If we execute Spark locally, the context creation can be used to Spark. It is also possible to launch Spark and the Python interpreter / notebook using the script named `pyspark`. In this case, the context will already be created (under the name `sc`) and a `ValueError` exception is raised when we try to create a second context. The exception has not effect so we simply catch it and display a warning message.

We can consult the dashboard of our Spark application at the link computed by the next cell.

In [ ]:
print("Spark app's dashboard link: {}".format(spark_helper.get_app_dashboard_url(sc)))

## 2. Creating a RDD

We will now create a RDD from text files representing the page visits data of Wikipedia. This data should be in the folder `data/pagecounts`. 

The notebook [0-Configuration.ipynb](0-Configuration.ipynb) can assist you in downloading the files.

In [ ]:
pagecounts = sc.textFile('data/pagecounts/*.gz')

A [pagecounts](https://dumps.wikimedia.org/other/pagecounts-raw/) file looks like this  
```
af Spesiaal:Onlangse_wysigings 3 101681
af Spesiaal:RecentChanges 2 2248
af Suid-Afrika 1 30698
af Tuisblad 14 155257 
af Varkgriep 4 42236
af Wikipedia 2 32796
```

It is a tabular file, where each line is a distinct entry and the columns represent
2. the project name (language);
3. the page title;
4. the number of requests;
5. the size of the content returned.

We can look at a few entries with the RDD's method `take` to get the first `K` elements of the dataset. Here, `K = 10`.

In [ ]:
first10 = pagecounts.take(10)
print(first10)

Since `take` returns a list, we can iterate on the result and print it "prettily".

In [ ]:
for item in first10:
    print(item)

## 3. Getting Help

At any moment, you can get help on a Python object using the `help()` function. For example, if we want to know more aboud the RDD's `take()` method.

In [ ]:
help(pagecounts.take)

## 4. Action on a Dataset

The `take()` method is one among multiple available *actions* we apply on a RDD. An exhaustive list of action is available at the following URL:
https://spark.apache.org/docs/latest/programming-guide.html#actions

In case where we do not want to leave the notebook tab, we can call `help()` directly on an RDD.

In [ ]:
help(pagecounts)

Among the available action, lets take the method `count()`. What does it return?

In [ ]:
pagecounts.count()

Each action apply on a RDD leads to the creation of one or many task and the production of a result. Every task executed in the same app can be visualised in the Spark's dashboard. In this interface, we can track the progress of a task, and check different performance measures on the task, for example its duration and cache statistics.

## 5. Dataset Transformation

If we display the 10 first element of our dataset that we retrieved earlier.

In [ ]:
first10

We realise that the RDD is composed of each line of input text files, but that is not possible to access to individual column. **Why?**

In [ ]:
first1 = pagecounts.first()
first1

The action `first()` as its name states, return the first entry of the dataset. We see that each entry is a single string. We will need to transform that first RDD in a second in order to divide each string in a list of 4 elements. To do this, we will use the method `split()` of Python string object.

We first test it on the dataset first entry.

In [ ]:
str.split(first1)

We now want to apply this transformation to every RDD's entry. The RDD's method `map(func)` returns a new distributed dataset formed by passing each element of the source through a function *func*.

In [ ]:
pagecounts_tab = pagecounts.map(str.split)

The evaluation of this transformation is *lazy*. Spark does not compute anything as long as a result is not requested by an action. To convince yourself, execute the preceding cell, then visit the Spark dashboard. You should see that no job have been added to the list.

In [ ]:
print("Spark app's dashboard link: {}/jobs".format(spark_helper.get_app_dashboard_url(sc)))

## 6. Caching a Dataset

When we expect to operate frequently on the same dataset, it can be useful to tell Spark to keep it in memory.

To do so, we use the `cache()` method.

In [ ]:
pagecounts_tab.cache()

The RDDs stored in memory are displayed in the **Storage** section of Spark web interface. Note that datasets are not loaded in memory until an action is called on them. 

In [ ]:
print("Spark app's dashboard link: {}/storage".format(spark_helper.get_app_dashboard_url(sc)))

To free memory used by cached RDD that we no longer need, we need to call the `unpersist()` method.

In [ ]:
pagecounts_tab.unpersist()

## 7. Filtering a Dataset

Since we now have a RDD that is easier to manipulate, we can star the analysis. First, we take interest in page s rom Wikipedia English. The following line filter the last RDD we created , eepsing nly the entries written in English.

Try to answer the following quiz before executing the cell:  
* What sort of argument takes the `filter()` method?
* What type is `x`?
* Is filter an action or a transformation?
* What does `filter()` return?

In [ ]:
pagecounts_en = pagecounts_tab.filter(lambda x: x[0] == "en")

#### Exercise 1
**Write the code to cache the new RDD in the following cell**.

We can now count the number of pages in English. The macro notebook `%time` will indicate how long it took Spark to count the number of entries in the RDD.

In [ ]:
%time pagecounts_en.count()

Since we told Spark to keep the dataset in memory, the time required to count the number of pages should be shorter for the second execution.

In [ ]:
%time pagecounts_en.count()

Since we committed an action on a cached RDD, it should now figure in the **Storage** section of our app's dashboard.

In [ ]:
print("Spark app's dashboard link: {}/storage".format(spark_helper.get_app_dashboard_url(sc)))

## 8. Reduction Operation

We are now interested in producing a bar chart of the number of pages request per language in our dataset. In order to do this, we will need to aggregate the number of requested pages for each language. This type of operation is called a *reduction*.

### 8.1 Transforming in Key-Value Pairs

First, we transform our dataset to only keep the language and the number of requests. Furthermore, the number of request field is a string, we therefore use `int()` to convert it in an integer.

In [ ]:
pagecounts_tuple = pagecounts_tab.map(lambda entry: (entry[0], int(entry[2])))

#### Exercise 2
**Take a look at the first 5 elements of the new RDD to confirm the transformation is correct.**

Spark provides functions to work with key-value pairs. In our new dataset, the key is the language and the number of requests the value. The `key()` method of RDD returns a new RDD composed only of the keys.

In [ ]:
pagecounts_tuple.keys().take(5)

A `values()` method is also available.

In [ ]:
pagecounts_tuple.values().take(5)

### 8.2 Filtering Unrelated Entries

As we observed, the key of our RDD sometimes has the form "xx.x". The project suffix ".x" corresponds to the project name. The following abbreviations are used:  
```
wikibooks: ".b"
wiktionary: ".d"
wikimedia: ".m"
wikipedia mobile: ".mw"
wikinews: ".n"
wikiquote: ".q"
wikisource: ".s"
wikiversity: ".v"
mediawiki: ".w"
```
Projects without a period and a following character are wikipedia projects. 

#### Exercise 3
**Design a filter to only keep the pages that are Wikipedia projects.**

### 8.3 Aggregating the Results by Key

We want to compute the number of pages equested for each language. In order to do this, we use the `reduceByKey()` method. As its name states, this function expect the RDD to be composed of key-value pairs.

When called on a dataset of $(K, V)$ pairs, `reduceByKey()` returns a dataset of $(K, V)$ pairs where the values for each key are aggregated using the given reduce function *func*, which must be of type $(V,V) \rightarrow V$.

Since we want the total number of pages er language, our aggregating function will be the addition.

In [ ]:
lang_pagecounts = pagecounts_filt.reduceByKey(lambda x, y: x + y).cache()

`reduceByKey()` is a transformation, therefore the result is a new RDD. To visualize the entire content of the latter, we can use the `collect()` method.

In [ ]:
lang_pagecounts.collect()

### 8.4 Computing the Most Frequent Languages

We now wish to determine the 5 most frequent languages requested during the period observed. In order to do this, multiple solutions are available to us.

#### 8.4.1. Sort locally

In [ ]:
top5 = sorted(lang_pagecounts.collect(), key=lambda x: x[1], reverse=True)[:5]
print(top5)

#### 8.4.2 Ask Spark to sort the dataset using the `sortBy()` method.

In [ ]:
top5 = lang_pagecounts.sortBy(keyfunc=lambda x: x[1], ascending=False).take(5)
print(top5)

#### 8.4.3 Ask Spark for the top 5 using the `top()` method.

In [ ]:
top5 = lang_pagecounts.top(5, lambda x: x[1])
print(top5)

#### Exercise 4
**Can you think of another method to get the same result?**

### 8.5 Bar Chart

Since we are in an interactive notebook, we can use Python plotting library `matplotlib` to plot our bar chart.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure()
ax = plt.subplot(111)
top5_t = list(zip(*top5))
ax.bar(range(len(top5_t[0])), top5_t[1], width=0.35, align="center")
ax.set_xticks(range(len(top5_t[0])))
ax.set_xticklabels(top5_t[0])
ax.set_xlabel("lang")
ax.set_ylabel(u"number of requests")

## 9. Ending the Analysis

Once the analysis is done, we need to tell Spark to free the resources and destroy the context using the `SparkContext` method `stop()`.

In [ ]:
sc.stop()

## 10. Recap

In this notebook, we used and learned about the following parts of 
**[Python Spark API](http://spark.apache.org/docs/latest/api/python/)**:
1. Import Spark Python module: 
**[`import pyspark`](http://spark.apache.org/docs/latest/api/python/pyspark.html)**
2. Create a SparkContext:
**[`pyspark.SparkContext()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext)**
2. Create a RDD from text files:
**[`SparkContext.textFile(path)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext.textFile)**
3. Take a the first *num* elements from a RDD: 
**[`Rdd.take(num)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.take)**
3. Count the number of elements in a RDD: 
**[`Rdd.count()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.count)**
4. Retrieve the first element of a RDD: 
**[`RDD.first()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.first)**
5. Apply a transformation on each element of a RDD:
**[`RDD.map(func)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map)**
4. Cache a RDD:
**[`RDD.cache()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.cache)**
5. Remove a RDD from memory: 
**[`RDD.unpersist()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.unpersist)**
5. Filter a RDD:
**[`RDD.filter(func)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.filter)**
6. Merge the values for each keys: 
**[`RDD.reduceByKey(func)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey)**
7. Get all elements of a RDD: 
**[`RDD.collect()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect)**
8. Sort the elements of a RDD:
**[`RDD.sortBy(func)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.sortBy)**
9. Get the top $N$ elements from a RDD:
**[`RDD.top(N)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.top)**
10. End the SparkContext:
**[`SparkContext.stop()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext.stop)**